In [1]:
import pandas as pd
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", False)
from sklearn.metrics import mean_absolute_percentage_error
from datetime import date, timedelta, datetime

In [2]:
# MODIFIABLE

Date_fin_train = '2021-10-31'    # <= Inférieur ou Egal
Nombre_jours_test = 180

In [3]:
# NON MODIFIABLE

Date_debut_test = pd.to_datetime(Date_fin_train)

Date_fin_test =  str(Date_debut_test + timedelta(Nombre_jours_test))
Date_debut_test = Date_fin_train
Date_fin_test

'2022-04-29 00:00:00'

In [4]:
df = pd.read_csv('../raw_data/data_preparation.csv', index_col=[0])
df

,Date,Code INSEE région,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW),Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore,TCO Thermique (%),TCH Thermique (%),TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),Column 30,YEAR,MONTH,DAY,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN,QV2M,RH2M,PRECTOTCORR,PS,WS10M,WS50M,sin_month,cos_month,week_day,sin_day,cos_day,bank_holiday,school_holidays,season,num_day,sin365_1,cos365_1,sin365_2,cos365_2,sin365_3,cos365_3
0,2013-01-01,4464,252371.0,15175.0,0.0,372.0,405.0,61254.0,-4.0,1808.0,173268.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,1,1,7.30,6.40,6.85,7.08,5.75,9.60,3.86,6.16,94.12,5.62,97.76,3.23,5.29,0.5,0.866025,1,0.781831,0.623490,1.0,1,1,1,0.017202,0.999852,0.034398,0.999408,0.068755,0.997634
1,2013-01-02,4464,280064.0,18369.0,0.0,1404.0,2655.0,70681.0,-12.0,1717.0,185143.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,1,2,4.36,1.79,3.08,4.10,8.23,10.05,1.83,4.46,84.50,0.08,98.04,4.65,6.87,0.5,0.866025,2,0.974928,-0.222521,0.0,1,1,2,0.034398,0.999408,0.068755,0.997634,0.137185,0.990545
2,2013-01-03,4464,285526.0,24755.0,0.0,626.0,2984.0,72282.0,0.0,2357.0,182412.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,1,3,5.46,0.91,3.19,3.78,10.59,11.96,1.37,4.15,73.75,0.00,99.42,2.33,3.35,0.5,0.866025,3,0.433884,-0.900969,0.0,1,1,3,0.051584,0.998669,0.103031,0.994678,0.204966,0.978769
3,2013-01-04,4464,286799.0,17085.0,0.0,1028.0,2570.0,68995.0,0.0,2988.0,194018.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,1,4,5.92,2.24,4.08,4.06,11.30,12.80,1.51,4.58,79.12,0.01,99.19,2.50,3.72,0.5,0.866025,4,-0.433884,-0.900969,0.0,1,1,4,0.068755,0.997634,0.137185,0.990545,0.271777,0.962360
4,2013-01-05,4464,267848.0,15479.0,0.0,1557.0,2898.0,62143.0,0.0,3051.0,182618.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,1,5,6.64,2.37,4.50,4.85,12.38,14.41,2.04,4.76,74.88,0.00,98.51,3.25,5.05,0.5,0.866025,5,-0.974928,-0.222521,0.0,1,1,5,0.085906,0.996303,0.171177,0.985240,0.337301,0.941397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,2022-05-11,4464,218296.0,34225.0,0.0,1909.0,10139.0,40944.0,-306.0,3115.0,128276.0,0.0,0.0,0.0,0.0,750.65,1257.37,0.0,0.0,41.85,1968.08,215.63,613.37,0.0,2022,5,11,18.10,10.98,14.55,18.10,10.06,23.51,13.45,8.36,65.50,0.00,98.39,2.50,3.04,0.5,-0.866025,2,0.974928,-0.222521,0.0,0,2,131,0.775859,-0.630907,-0.978989,-0.203914,0.399258,-0.916838
3413,2022-05-12,4464,209770.0,47313.0,0.0,1695.0,15998.0,40585.0,-306.0,3177.0,101324.0,0.0,0.0,0.0,0.0,1075.23,1738.16,0.0,0.0,38.92,1747.49,359.01,967.83,0.0,2022,5,12,18.13,10.08,14.11,18.05,10.38,23.49,13.11,7.81,61.56,0.00,98.51,2.45,3.00,0.5,-0.866025,3,0.433884,-0.900969,0.0,0,2,132,0.764891,-0.644159,-0.985424,-0.170118,0.335276,-0.942120
3414,2022-05-13,4464,201061.0,74913.0,0.0,1215.0,18002.0,41576.0,0.0,3561.0,61809.0,0.0,0.0,0.0,0.0,1795.44,2752.11,0.0,0.0,29.28,1252.63,419.87,1089.04,0.0,2022,5,13,18.01,10.59,14.30,18.40,10.09,23.48,13.39,8.24,64.94,0.10,98.33,2.34,3.11,0.5,-0.866025,4,-0.433884,-0.900969,0.0,0,2,133,0.753698,-0.657221,-0.990692,-0.136120,0.269707,-0.962943
3415,2022-05-14,4464,194195.0,73265.0,0.0,266.0,19831.0,40031.0,-110.0,3245.0,57672.0,0.0,0.0,0.0,0.0,1816.54,2691.58,0.0,0.0,6.62,274.19,479.62,1199.69,0.0,2022,5,14,19.47,14.94,17.20,20.69,13.58,26.39,12.80,10.93,77.94,0.83,98.08,2.03,2.70,0.5,-0.866025,5,-0.974928,-0.222521,0.0,0,2,134,0.742281,-0.670089,-0.994788,-0.101962,0.202861,-0.979208


#  Pipeline

In [5]:
preproc_MinMaxScaler = make_pipeline(MinMaxScaler())
preproc_StandardScaler = make_pipeline(StandardScaler()) 
preproc_RobustScaler = make_pipeline( RobustScaler())

In [6]:
min_max = ['t - 1', 't - 2', 't - 3', 't - 4', 't - 5', 't - 6', 't - 7', 't - 8', 't - 9',
       't - 10', 't - 11', 't - 12', 't - 13', 't - 14', 't - 15', 't - 16',
       't - 17', 't - 18', 't - 19', 't - 20', 't - 21', 't - 22', 't - 23',
       't - 24', 't - 25', 't - 26', 't - 27', 't - 28', 't - 29', 't - 30']
stand = ['T2MDEW','T2M_RANGE']
robus = ['T2M', 'T2MWET', 'TS', 'T2M_MAX', 'T2M_MIN','QV2M', 'RH2M', 'PRECTOTCORR', 'PS', 
         'WS10M', 'WS50M']

In [7]:
preproc = make_column_transformer(
    (preproc_MinMaxScaler, min_max),
    (preproc_StandardScaler, stand),
    (preproc_RobustScaler, robus),
    remainder="passthrough")

preproc

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['t - 1', 't - 2', 't - 3', 't - 4', 't - 5',
                                  't - 6', 't - 7', 't - 8', 't - 9', 't - 10',
                                  't - 11', 't - 12', 't - 13', 't - 14',
                                  't - 15', 't - 16', 't - 17', 't - 18',
                                  't - 19', 't - 20', 't - 21', 't - 22',
                                  't - 23', 't - 24', 't - 25', 't - 26',
                                  't - 27', 't - 28', 't - 29', 't - 30']),
                                ('pipeline-2',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 ['T2MDEW', 'T2M_RANGE']),

In [8]:
model_xgb = XGBRegressor(colsample_bytree = 1, 
                         gamma= 0.1, 
                         max_depth= 16, 
                         min_child_weight= 6, 
                         n_estimators= 100,
                         learning_rate=0.1)

In [9]:
pipeline_xgb_final = make_pipeline(preproc,model_xgb)
pipeline_xgb_final

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['t - 1', 't - 2', 't - 3',
                                                   't - 4', 't - 5', 't - 6',
                                                   't - 7', 't - 8', 't - 9',
                                                   't - 10', 't - 11', 't - 12',
                                                   't - 13', 't - 14', 't - 15',
                                                   't - 16', 't - 17', 't - 18',
                                                   't - 19', 't - 20', 't - 21',
                                                   't - 22', 't - 23', 't - 24',
                              

# Création data frame

In [10]:
df_train = df[ (df["Date"] <= Date_fin_train)]
df_test = df[ (df["Date"] > Date_debut_test) & (df["Date"] <= Date_fin_test)]

In [11]:
X_train = df_train.drop(columns=['Date', 'Code INSEE région', 'Consommation (MW)', 'Thermique (MW)',
       'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)',
       'Pompage (MW)', 'Bioénergies (MW)', 'Ech. physiques (MW)',
       'Stockage batterie', 'Déstockage batterie', 'Eolien terrestre',
       'Eolien offshore', 'TCO Thermique (%)', 'TCH Thermique (%)',
       'TCO Nucléaire (%)', 'TCH Nucléaire (%)', 'TCO Eolien (%)',
       'TCH Eolien (%)', 'TCO Solaire (%)', 'TCH Solaire (%)', 'Column 30','YEAR', 'MONTH', 'DAY', 'season', 'num_day'])

y_train = df_train['Consommation (MW)']

In [12]:
X_test = df_test.drop(columns=['Date', 'Code INSEE région', 'Consommation (MW)', 'Thermique (MW)',
       'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)',
       'Pompage (MW)', 'Bioénergies (MW)', 'Ech. physiques (MW)',
       'Stockage batterie', 'Déstockage batterie', 'Eolien terrestre',
       'Eolien offshore', 'TCO Thermique (%)', 'TCH Thermique (%)',
       'TCO Nucléaire (%)', 'TCH Nucléaire (%)', 'TCO Eolien (%)',
       'TCH Eolien (%)', 'TCO Solaire (%)', 'TCH Solaire (%)', 'Column 30','YEAR', 
        'MONTH', 'DAY', 'season', 'num_day',
       ])

y_test = df_test['Consommation (MW)']

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3226, 26)
(3226,)
(180, 26)
(180,)


In [14]:
X_test[['t - 26', 't - 27', 't - 28', 't - 29', 't - 30']]

KeyError: "None of [Index(['t - 26', 't - 27', 't - 28', 't - 29', 't - 30'], dtype='object')] are in the [columns]"

In [15]:
X_test.columns

Index(['T2M', 'T2MDEW', 'T2MWET', 'TS', 'T2M_RANGE', 'T2M_MAX', 'T2M_MIN',
       'QV2M', 'RH2M', 'PRECTOTCORR', 'PS', 'WS10M', 'WS50M', 'sin_month',
       'cos_month', 'week_day', 'sin_day', 'cos_day', 'bank_holiday',
       'school_holidays', 'sin365_1', 'cos365_1', 'sin365_2', 'cos365_2',
       'sin365_3', 'cos365_3'],
      dtype='object')

# Fit model

In [ ]:
pipeline_xgb_final.fit(X_train, y_train)

In [ ]:
X_test.shape

# Prédictions

In [ ]:
predictions = []
X_test_i = pd.DataFrame([X_test.iloc[0,26:]])

X_test_0 = pd.DataFrame([X_test.iloc[0,:]])

y_i= pipeline_xgb_final.predict(X_test_0)

predictions.append(y_i[0])

In [ ]:
for i in range (1,Nombre_jours_test):
    X_test_features = pd.DataFrame([X_test.iloc[i,:26]])

    
    X_test_i.iloc[0,1:] = X_test_i.iloc[0,:-1]
    X_test_i.iloc[0,0] = y_i[0]
    
    X_test_decal = pd.DataFrame(X_test_i)
    
 
    X_test_pred = pd.merge(left = X_test_features.reset_index(drop = True), right = X_test_decal.reset_index(drop = True),
                left_index = True, right_index = True)
    
    y_i= pipeline_xgb_final.predict(X_test_pred)
    

    predictions.append(y_i[0])

In [ ]:
len(predictions)

In [ ]:
X_train.shape

# Lecture prédictions

In [ ]:
type(y_test)

In [ ]:
y_test_list = y_test.tolist()

In [ ]:
y_test_list

In [ ]:
MAPE = mean_absolute_percentage_error(y_test, predictions)
MAPE

In [ ]:
plt.figure(figsize=(16,20))#figsize=(10,6)
plt.plot(y_test_list, color='blue', label='consommation réel')
plt.plot(predictions, color='red', label='prédiction de consommation')
plt.title('Prédiction de consommation')
plt.xlabel('Date')
plt.ylabel('Consommation')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,20))#figsize=(10,6)
plt.plot(y_test_list, color='blue', label='consommation réel')
plt.plot(predictions, color='red', label='prédiction de consommation')
plt.title('Prédiction de consommation')
plt.xlabel('Date')
plt.ylabel('Consommation')
plt.legend()
plt.show()

In [ ]:
print(f'Durée entrainement = {X_train.shape[0]} // Date fin de train = {Date_debut_test}  // Nombre jours de prédictions = {Nombre_jours_test} // MAPE = {MAPE}')

In [ ]:
# Durée entrainement = 3255 // Date fin de train = 2021-12-31  // Nombre jours de prédictions = 14 // MAPE = 0.020387443598371847
# Durée entrainement = 2159 // Date fin de train = 2018-12-31  // Nombre jours de prédictions = 720 // MAPE = 0.04392174749255917
# Durée entrainement = 3133 // Date fin de train = 2021-08-31  // Nombre jours de prédictions = 14 // MAPE = 0.019562152727553097
# Durée entrainement = 3194 // Date fin de train = 2021-10-31  // Nombre jours de prédictions = 14 // MAPE = 0.02608112365233331
# Durée entrainement = 3345 // Date fin de train = 2022-03-31  // Nombre jours de prédictions = 14 // MAPE = 0.04344918094182426
# Durée entrainement = 3194 // Date fin de train = 2021-10-31  // Nombre jours de prédictions = 180 // MAPE = 0.02717062537406969